<a href="https://colab.research.google.com/github/samia-boubaya/project-01-escape-room-game/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **PROJECT 01: ESCAPE ROOM GAME** - IRONHACK DATA ANALYTICS

Welcome to escape room quest

You wake up in a couch in a misteryous house with locked doors.
Your mission: Find the keys, unlock each room, and escape to freedom!

In this notebook we start by building the data structures that define the game world, which are: rooms, objects, keys and doors.

**Definition of an escape room:**

The briefing room of an escape room in Tampere, Finland
Escape rooms are inspired by escape room video games, live-action role-playing, point-and-click adventure games, puzzle hunts, interactive theater, and haunted houses. They are also referred to as "room escapes", "escape games", "exit games", or "live escapes".


------
------
------
------

# **FINAL GAME PRODUCT**

----
## **ALL EXTRA FEATURES:**

----
✅ GAME TRIGGERS (RODRIGO) 

✅ CLOCK | COUNTDOWN (SAMIA) 

✅ HEALTH (REYNOLD) 

✅ TAKE A BREAK (PAULINA)

----

---
## **DATA:**


#### LISTS: ACTIONS, SPACES, ITEMS, DOORS, KEYS

In [ ]:
# LIST : actions available to a player
actions = ['explore', 'examine', 'unlock door', 'navigate', 'restart', 'quit','play']
print("Player's actions in the game :",actions)

# LIST : spaces
spaces = ['game room', 'bedroom 1', 'bedroom 2', 'living room','outside']
print("\nSpaces in the game :",spaces)

# LIST : items
items = ['couch', 'piano', 'queen bed', 'double bed', 'dresser', "dining table"]
print("\nItems in the game :",items)

# LIST : doors
doors = ['door A', 'door B', 'door C', 'door D']
print("\nDoors in the game :",doors)

# LIST : keys
keys = ['key door A', 'key door B', 'key door C', 'key door D']
print("\nKeys in the game :",keys)


#### DICT : GAME AREAS

In [ ]:
# Each key in this dictionary represents a room; each value is a list of objects, doors and keys

# Dictionary : GAME AREAS
game_areas = {
    "game room" : ['couch', 'piano', 'door A', 'key door A'],
    "bedroom 1": ['queen bed', 'door A', 'door B', 'door C', 'key door B'],
    "bedroom 2": ['double bed', 'door B', 'dresser', 'key door D', 'key door C'],
    "living room": ["dining table", 'door C', 'door D'],
    "outside": ["freedom"]
}

# List : dictionary Keys
spaces = game_areas.keys()
print("GAME AREAS Dictionary:")
print("Dictionary Keys:\n",spaces,"\n")

# Print dictionary organised
print("Dictionary:\n{   key   :   value     }")
for i in spaces:
  print(i,":",game_areas[i])

#### DICT : GAME STATE

In [ ]:
game_state = {
    'space path' :[], # player current space to navigate and make a space path
    'item path' :[], # to select an item to examine for key; and make an item path
    'door path':[], # to select a door in currentspace and track unlocked doors already and locked ones
    'inventory':[], # to store found keys
    'play time':0, # import library time for time limit max 40min
    'health':100, # health scores whenever the countdown repeats we lose 25 points
}
print(game_state)

----
## ***EXTRA FEATURES COMBINED***

----
#### MUST INSTALL & DEFINE EXTRA FEATURES FUNCTIONS

In [ ]:
!pip install pillow
!pip install playsound==1.2.2
!pip install pyobjc

In [ ]:
import os
import time
import threading
import platform
from PIL import Image
from IPython.display import display, clear_output
import requests
import hashlib
from datetime import datetime, timedelta
import sys

try:
    from playsound import playsound
except ImportError:
    raise ImportError("Please install playsound with: pip install playsound==1.2.2")

# =============================================
# ⚙️ GAME STATE
# =============================================
# Dictionary : GAME STATE
game_state = {
    "space path" :[], # player current space to navigate and make a space path
    "item path" :[], # to select an item to examine for key; and make an item path
    "door path":[], # to select a door in currentspace and track unlocked doors already and locked ones
    "inventory":[], # to store found keys
    "play time":0, # import library time for time limit max 40min
    "health":100, # health scores whenever the countdown repeats we lose 25 points

}

total_play_time = 0
BANNER_WIDTH = 70

# =============================================
# ⚙️ TRIGGERS CONFIGURATION
# =============================================
TRIGGERS = {
    "space": {
        "sound": "https://drive.google.com/uc?export=download&id=1TH3ssE0tVnfKySil0q55grCoiFpCxFO7",
        "image": "https://drive.google.com/uc?export=download&id=1LjUpXnqjCmpvhJZ8iXk-3d1qXR-plCPA",
    },
    "examine": {
        "sound": "https://drive.google.com/uc?export=download&id=1_FbFeK51rtIMWg9yQovpurNGHHWukYoD",
        "image": "https://drive.google.com/uc?export=download&id=1igPXeO1Ro-UUL1tMQsMAQqOZE1kPundT",
    },
    "found key": {
        "sound": "https://drive.google.com/uc?export=download&id=1MhGR7E9YXp7LrzKaSMlCGd211I18P6Iy",
        "image": "https://drive.google.com/uc?export=download&id=1hcyiCkmmchsGZ2ERsx8PMdEN3kh--tO5",
    },
    "not found key": {
        "sound": "https://drive.google.com/uc?export=download&id=17v52HWgz_jCHFITeCV1oyOOC4TV_gyzT",
        "image": "https://drive.google.com/uc?export=download&id=1UV-xsA9ipAuN6SwhcBCvtET9nDM4YRal",
    },
    "door locked": {
        "sound": "https://drive.google.com/uc?export=download&id=1-BwRmk_3U6TRESsPja5s26ECB8GYcqOU",
        "image": "https://drive.google.com/uc?export=download&id=1drL22OHO91OgjpT942g2nrmgRZcr0gYa",
    },
    "quiz": {
        "sound": "https://drive.google.com/uc?export=download&id=1wbj__J8nYqmeOh1gCmZVaWWqMbF4MNyp",
        "image": "https://drive.google.com/uc?export=download&id=1vYJV2WLgFkuDbXaITMFshGfabjs1A2Qs",
    },
    "door unlocked": {
        "sound": "https://drive.google.com/uc?export=download&id=16bhcdxVRzIRmNO0ztLaeQZYHBLPVlB8P",
        "image": "https://drive.google.com/uc?export=download&id=1TXSjKZXC_K0c4gXrQlTY20u_X2aEVQ1_",
    },
    "footstep": {
        "sound": "https://drive.google.com/uc?export=download&id=1mlkfF9RW6uhKMeBqixYZlo5w-dVJ_1Uo",
        "image": "https://drive.google.com/uc?export=download&id=16snfruJnkMN8fkxL_fxDzwCaNhNdRroN",
    },
    "outside": {
        "sound": "https://drive.google.com/uc?export=download&id=13vxJyjMuAz0hHfoaqLIrmOEbf5IN8BGW",
        "image": "https://drive.google.com/uc?export=download&id=1fVIHmH3PCfJVposo9749Sgo4q5dLwQea",
    },
}

os.makedirs("cache", exist_ok=True)

# =============================================
# 🧠 UTILITIES
# =============================================

def download_file(url: str, name: str) -> str | None:
    """Download file if not cached."""
    _, ext = os.path.splitext(name.lower())
    subfolder = "sounds" if ext in [".mp3", ".wav", ".ogg"] else "images"
    target_dir = os.path.join(os.getcwd(), "assets", subfolder)
    os.makedirs(target_dir, exist_ok=True)

    url_hash = hashlib.md5(url.encode()).hexdigest()[:8]
    base, ext = os.path.splitext(name)
    unique_name = f"{base}_{url_hash}{ext}"
    local_path = os.path.join(target_dir, unique_name)

    if os.path.exists(local_path):
        return local_path

    print(f"⬇️ Downloading {name} → {target_dir}/")
    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        with open(local_path, "wb") as f:
            f.write(response.content)
        return local_path
    except Exception as e:
        print(f"⚠️ Download error: {e}")
        return None

# =============================================
# 🔊 SOUND PLAYER
# =============================================
_event_lock = threading.Lock()

def _play_once(path):
    if platform.system() == "Darwin":
        os.system(f"afplay '{path}' >/dev/null 2>&1")
    elif platform.system() == "Linux":
        os.system(f"mpg123 '{path}' >/dev/null 2>&1")
    else:
        playsound(path, block=True)

def play_sound(url: str, trigger_name: str, loop_duration: int = None, ambient=False):
    local_file = download_file(url, f"{trigger_name}.mp3")
    if not local_file:
        return

    def _play():
        try:
            start = time.time()
            while True:
                if not ambient:
                    with _event_lock:
                        _play_once(local_file)
                else:
                    _play_once(local_file)

                if not loop_duration or (time.time() - start > loop_duration):
                    break
        except Exception as e:
            print(f"⚠️ Sound error: {e}")

    threading.Thread(target=_play, daemon=True).start()

# =============================================
# 🖼️ IMAGE DISPLAY
# =============================================
def show_image_inline(url: str, display_time: int = 5000):
    if not url:
        return
    try:
        local_file = download_file(url, "image.gif")
        if not local_file:
            return
        img = Image.open(local_file)
        img = img.resize((400, 300), Image.Resampling.LANCZOS)
        display(img)

        def safe_clear():
            try:
                clear_output(wait=True)
            except LookupError:
                pass
        threading.Timer(display_time / 1000, safe_clear).start()
    except Exception as e:
        print(f"Image error: {e}")

# =============================================
# 🚀 MERGED TRIGGER FUNCTION
# =============================================
def trigger_event(trigger_name: str, display_time: int = 5000):
    """Handles messages, state updates, images, and sounds."""
    trigger_name = trigger_name.lower().strip()
    
    # State messages
    messages = {
        "space": "\n🚀 You move to a new space!",
        "examine": "\n🔍 You examine the item...",
        "found key": "\n🗝️ You found a key!",
        "not found key": "\n❌ Not found key here...",
        "door locked": "\n🚪 The door is locked.",
        "quiz": "\n❓ A quiz challenge appears!",
        "footstep": "\n👣 You hear footsteps nearby...",
        "outside": "\n🌳 You step outside."
    }
    print(messages.get(trigger_name, f"\n⚠️ Unknown trigger: {trigger_name}"))


#######################################################################################

    # Play visuals and sound
    data = TRIGGERS.get(trigger_name)
    if data:
        if data.get("sound"):
            play_sound(data["sound"], trigger_name)
        if data.get("image"):
            show_image_inline(data["image"], display_time)

# =============================================
# 🛡️ HEALTH SYSTEM
# =============================================
def print_banner(health):
    def center_line(text):
        padding = (BANNER_WIDTH - 2 - len(text)) // 2
        return f"{' ' * padding}{text}{' ' * (BANNER_WIDTH - 2 - len(text) - padding)}"
    
    print("-" * BANNER_WIDTH)
    if health == 100:
        print(center_line("💪 You feel strong!"))
        print(center_line("Ready to face the next challenge!"))
    else:
        print(center_line("TIME'S UP! Restarting countdown..."))
    print(center_line(f"Current Health: {health}"))
    print("-" * BANNER_WIDTH)

def update_health():
    game_state['health'] = max(game_state['health'] - 25, 0)

def restore_health():
    if game_state['health'] == 0:
        choice = input("💉 Health is 0! Restore to 100? (y/n): ").strip().lower()
        if choice == 'y':
            game_state['health'] = 100
            print("❤️ Health restored to 100!")
            return True
        else:
            print("☠️ Game over.")
            return False
    return True

# =============================================
# ⏳ COUNTDOWN
# =============================================
def start_countdown(minutes):
    global total_play_time
    end_time = datetime.now() + timedelta(minutes=minutes)
    
    while datetime.now() < end_time:
        now = datetime.now()
        remaining = end_time - now
        mins, secs = divmod(int(remaining.total_seconds()), 60)
        print(f"\r🕰️ {now.strftime('%H:%M:%S')} | ⏳ {mins:02}:{secs:02}", end="")
        time.sleep(0.1)
    
    print("\r🕰️ Time's up!               ")
    update_health()
    total_play_time += minutes
    print_banner(game_state['health'])
    return restore_health()

# =============================================
# 🎮 MAIN FUNCTION
# =============================================
def play_game(countdown_minutes, trigger, image_minutes=1):
    """Play one full session with countdown and image display duration."""
    #print(f"\n🎮 Starting game with trigger '{trigger}' for {countdown_minutes} min\n")
    trigger_event(trigger, display_time=int(image_minutes * 60 * 1000))
    if not start_countdown(countdown_minutes):
        return
    #print(f"\n✅ Session finished for trigger '{trigger}'!\n")

# =============================================
# 🟢 RUN TESTS
# =============================================
#if __name__ == "__main__":
    
    # trigger='space' (5 minutes) # still not doing 5min
    #play_game(countdown_minutes=5, trigger='space', image_minutes=5)
    
    # trigger='examine' (6 seconds)
    #play_game(countdown_minutes=0.1, trigger='examine', image_minutes=0.1)
    
    # trigger='not found key' (6 seconds)
    #play_game(countdown_minutes=0.1, trigger='not found key', image_minutes=0.1)

    # trigger='found key' (6 seconds)
    #play_game(countdown_minutes=0.1, trigger='found key', image_minutes=0.1)
    
    # trigger='door locked' (6 seconds)
    #play_game(countdown_minutes=0.1, trigger='door locked', image_minutes=0.1)

    # trigger='door unlocked' (6 seconds)
    #play_game(countdown_minutes=0.1, trigger='door unlocked', image_minutes=0.1)
    
    # trigger='footstep' (6 seconds)
    #play_game(countdown_minutes=0.1, trigger='footstep', image_minutes=0.1)

    # trigger='outside' (6 seconds)
    #play_game(countdown_minutes=0.1, trigger='outside', image_minutes=0.1)
    

#### FUNCTION : PLAY_TRIGGER(space) ✅ 
{runs all interactive extras: text + image + sound+clock + countdown + HP + take a break}

In [ ]:
def play_trigger(trigger: str):
    if trigger in TRIGGERS.keys():
        if __name__ == "__main__":    
        
            if trigger=="space":
                play_game(countdown_minutes=5, trigger='space', image_minutes=5)
            
            elif trigger=="examine":
                trigger_event('examine')
            
            elif trigger=="not found key":
                trigger_event('not found key')
            
            elif trigger=="found key":
                trigger_event('found key')
            
            elif trigger=="door locked":
                trigger_event('door locked')

            elif trigger=="door unlocked":
                trigger_event('door unlocked')
                
            elif trigger=="quiz":
                trigger_event('quiz')

            elif trigger=="footstep":
                trigger_event('footstep')

            elif trigger=="outside":
                trigger_event('outside')
            
    
            else:
                print("trigger not in dictionary TRIGGERS!!!")
               
    

---
## ***PLAYER_INPUT = ?***

#### FUNCTION : ITEMS_IN(space:list) 
✅ return LIST OF ITEMS in current space

In [ ]:
def items_in(space: list):
    space_items=[]
    for item in items:
        if item in space:
            #print("-", item)
            space_items.append(item)
    return space_items


#### FUNCTION : EXPLORE (space) 
✅ return LIST OF ITEMS and select item

In [ ]:
def explore(space:str):
    print(f"You are exploring the {space}. You see these items:")
    space_items=[]
    for item in items:
        if item in game_areas[space]:
            print("-", item)
            space_items.append(item)
     
    select_item = input(f"Availiable items:({', '.join(space_items)}) | Select an item in {space} to examine: ").lower()
    while select_item not in space_items:
        select_item = input(f"Availiable items:({', '.join(space_items)}) | Select an item in {space} to examine: ").lower()
        breakpoint
    return space_items, select_item
        

space='game room'
explore(space)

#### FUNCTION : SEL_KEY(space) ✅ keys in current space

In [ ]:
def sel_key(space:str):
    print(f"You are exploring the {space}. You see these doors:")
    space_keys=[]
    for key in keys:
        if key in game_areas[space]:
            print("-", key)
            space_keys.append(key)
     
    select_key = input(f"Find the following keys:({', '.join(space_keys)}) | Select a key in {space} to store : ")
    while select_key not in space_keys:
        select_key = input(f"Find the following keys:({', '.join(space_keys)}) | Select a key in {space} to store: ")
        breakpoint
    print(f"You have selected the {select_key} from the {space}")

    return space_keys, select_key
sel_key("bedroom 2")


#### FUNCTION : CHECK_KEY(item, key) ✅ returns boolean result for searching for key in item

In [ ]:

#check for key in selected item in current space
# We use this function inside examine() to check inventory if there is a key  or not 
def check_key(item,space):
    space_keys, select_key = sel_key(space)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    #if found key-------------------------------------------------------------------------------------------------------
    if result!='': 
        #play_trigger('found key')
        print(f'You found the {select_key} in the {space}')
        game_state['inventory'].append(select_key)
        result = True

    #elif not found key--------------------------------------------------------------------------------------------------
    else:
        #play_trigger('not found key')
        print(f"PWAN~ You didn't find any key in the {select_key}.")
        result=False 

    #else key in inventory-----------------------------------------------------------------------------------------
    for key in game_state['inventory']:  
        if key in space_keys:
            print(f'You already have the key in your inventory: {game_state["inventory"]}')
            result = True
            #play_trigger('inventory')   # to show image of inventory full with key, and not repeat update it
    
    
    return result # returns a list of items in space

#### FUNCTION : EXAMINE (space) ✅ return TRUE or False depends on found key or not found key

In [26]:
def examine(space:str):
    # We show a list of items in space
    #play_trigger("examine")

    # select an item to examine from explore
    space_items, select_item = explore(space) 
    print(f"You carefully examine the {select_item}...")
    
    # ITEMS in SPACE --------------------------------------------------------------
    if select_item in space_items: 
        
        # MAKE ITEM KEY DICT
        # GAME ROOM ITEMS ----------------------------------------------------------
        if select_item == 'couch':
            result_examine = False
            print(f"you didn't find any key in the {select_item} :/")
        elif select_item == 'piano':
            key = 'key door A'
            result_examine = True
            print(f"you found the {key} in the {select_item}!")

        # BEDROOM 1 ITEM-------------------------------------------------------------
        elif select_item == 'queen bed':
            key = 'key door B'
            result_examine = True
            print(f"you found the {key} in the {select_item}!")

        # BEDROOM 2 ITEMS-------------------------------------------------------------
        elif select_item == 'double bed':
            key = 'key door C'
            result_examine = True
            print(f"you found the {key} in the {select_item}!")
        elif select_item == 'dresser':
            key = 'key door D'
            result_examine = True
            print(f"you found the {key} in the {select_item}!")

        # LIVING ROOM ITEMS---------------------------------------------------------
        elif select_item == 'dinning table':
            result_examine = False
            print(f"you didn't find any key in the {select_item} :/")
        # UPDATE ITEM PATH & INVENTORY---------------------------------------------------------
        #game_state["item path"].append(select_item)
        #game_state["inventory"].append(key)
        # Re-INPUT------------------------------------------------------------------
    select_key=key
    return result_examine, select_key

#######################################################################################


#--------------------------------------------------------------------------------------------------------

game_state.clear()
space='game room'

examine(space)


You are exploring the game room. You see these items:
- couch
- piano
You carefully examine the piano...
you found the key door A in the piano!


(True, 'key door A')

In [ ]:
def check_inventory():
    result, key, space_keys= examine(space)

    #if found key-------------------------------------------------------------------------------------------------------
    if result== True: 
        # key in inventory
        for key in game_state['inventory']:
            if key in space_keys:
                #play_trigger('found key')
                #print(f'You found the {select_key} in the {space}')
                #game_state['inventory'].append(select_key)
        

    #elif not found key--------------------------------------------------------------------------------------------------
    else:
        #play_trigger('not found key')
        #print(f"PWAN~ You didn't find any key in the {select_key}.") 

    #else key in inventory-----------------------------------------------------------------------------------------
    for key in game_state['inventory']:  
        if key in space_keys:
            print(f'You already have the key in your inventory: {game_state["inventory"]}')
            result = True
            #play_trigger('inventory')   # to show image of inventory full with key, and not repeat update it
    

    #else key in inventory-----------------------------------------------------------------------------------------
    for key in game_state['inventory']:  
        if key in space_keys:
            #if found key-------------------------------------------------------------------------------------------------------
            if result==True: 
                #play_trigger('found key')

                print(f'You already have the key in your inventory: {game_state["inventory"]}')
                result = True
            #play_trigger('inventory')   # to show image of inventory full with key, and not repeat update it
    

#### FUNCTION : DOORS_IN (space) ✅ returns list of doors in current space

In [ ]:
def doors_in(space):
    print(f"You are trying to get out of the {space}. You see these doors:")
    space_doors=[]
    for door in doors:
        if door in game_areas[space]:
            print("-", door)
            space_doors.append(door)
    
    select_door = input(f"Availiable doors:({', '.join(space_doors)}) | Select a door in {space} to try unlock: ").lower()
    while select_door not in space_doors:
        select_door = input(f"Availiable items:({', '.join(space_doors)}) | Select a door in {space} to try unlock: ").lower()
        breakpoint
    print(f"You have selected the {select_door} in the {space}")

    return space_doors, select_door #for testing

#doors_in('bedroom 2')


#### FUNCTION : UNLOCK DOOR (inventory, action, space, door) ✅ 
return True or False door unlocked or door locked

In [ ]:
def unlock_door(space:str):
    select_door = input(f"Availiable doors:({', '.join(space_doors)}) | Select an door in the {space} to unlock: ").lower()
    if select_door in doors_in(space):
        print(f'You are trying to unlock the {select_door} in the {space}.')
        door = input("Enter the door you want to try to unlock: ").lower()
    select_door = input(f"Availiable doors:({', '.join(space_doors)}) | Select an door in the {space} to unlock: ").lower()

    # check if inventory contains a key matching the door
    for key in game_state['inventory']:
        if door.lower() in key.lower():
            match_door_key = True
            print(f"The {key} in your inventory matches the {select_door}!")                
            breakpoint

    if key in game_state['inventory']:
        print(f"You can use the {key} to unlock {select_door}.")
        game_state['door path'].append(door)
        match_door_key = True
    else:
        print(f"The {select_door} is locked. You need a matching key.")
        trigger_event('door locked')
        match_door_key = False
    space_doors = doors_in(space)
    select_door = input(f"Availiable doors:({', '.join(space_doors)}) | Select an item in {space} to examine: ").lower()


    return select_door, match_door_key  # exit after one attempt

unlock_door('game room')

#### FUNCTION : QUIZ (inventory, door) ✅

In [ ]:
# quiz(inventory, door)

def quiz(inventory: list, door:str):
    # Quiz function to unlock doors
    for key in inventory:
        if door in key:
            print(f"You have the key to {door} in your inventory ~ you can proceed to unlock!")
            play_trigger("quiz")
            print(f"\n...  Trick or treat answer this quiz to unlock {door}!")
            
            # QUIZ(inventory, door)  #######################################################################################################################
          
            # quiz(game_state['inventory'], 'door A')
            if door == 'door A': 
                
                print("Question: What is the primary function of Door A, as suggested by its location in the floor plan?")
                print("A) To access the outdoors.")
                print("B) To provide entry or exit to a specific room.")
                print("C) To serve as a decorative element.")

                answer = input("Enter your choice (A, B, or C): ").upper()
                print(f"You chose the answer: {answer}")
                if answer == "B":
                    print("Correct! Door A is most likely for entering or exiting a room.")
                    trigger_event('door unlocked')
                    quiz_answer=True # return true to finally unlock door
                    return quiz_answer
                else:
                    print("\Incorrect. Try another time!")
                    quiz_answer=False
                    player_action('play') # call main function
                    
            # quiz(game_state[inventory], 'door B')
            elif door == 'door B': 
                print("Question: Considering the layout, which room is Door B most likely connected to?")
                print("A) The Game Room")
                print("B) Bedroom 1")
                print("C) The Outdoors")

                answer = input("Enter your choice (A, B, or C): ").upper()
                if answer == "B":
                    print("Correct! Based on the plan, Door B likely leads to Bedroom 1.")
                    quiz_answer=True # return true to finally unlock door
                    return quiz_answer
                else:
                    print("\nIncorrect. Try another time!")
                    trigger_event('door unlocked')
                    quiz_answer=False
                    player_action('play') # call main function
                    
            # quiz(game_state[inventory], 'door C')
            elif door == 'door C':
                print("Question: If you wake up on the couch, and the key to Door C is found nearby, what is the most logical room Door C leads to, considering the floor plan?")
                print("A) The Game Room")
                print("B) Bedroom 2")
                print("C) The Outdoors")
                
                answer = input("Enter your choice (A, B, or C): ").upper()
                if answer == "C":
                    print("Correct! It makes sense that Door C might lead outside.")
                    quiz_answer=True # return true to finally unlock door
                    return quiz_answer
                else:
                    print("\nIncorrect. Consider the layout again!")
                    trigger_event('door unlocked')
                    quiz_answer=False
                    player_action('play') # call main function
            
            # quiz(game_state[inventory], 'door D')
            elif door == 'door D': 
                print("Question: Considering the floor plan, and the fact you woke up on the couch, where is Door D most likely located?")
                print("A) In the Game Room")
                print("B) In Bedroom 1")
                print("C) Not visible on the plan")

                answer = input("Enter your choice (A, B, or C): ").upper()
                if answer == "C":
                    print("Correct! Since Door D isn't shown, it's not visible on the plan.")
                    quiz_answer=True # return true to finally unlock door
                    return quiz_answer
                else:
                    print("\nIncorrect. Maybe Door D is a secret door?")
                    trigger_event('door unlocked')
                    quiz_answer=False
                    player_action('play') # call main function
                    
            #####################################################################################################################################
    return quiz_answer


#### PLAYER_INPUT = NAVIGATE ✅

In [ ]:
def navigate(space, door):
    """Move from one space to another through a door if possible."""
    print(f"\nNavigating through {door}...")

    # Validate that both the space and door exist in the game world
    if door not in doors or space not in spaces:
        print("Invalid door or space!")
        player_action('play')
        return

    # Check if door was already unlocked
    if door not in game_state['door path']:
        print(f"The {door} is locked. You need to unlock it first!")
        return

    # Determine possible connections based on game_areas
    for next_space, contents in game_areas.items():
        if door in contents and next_space != space:
            game_state['space path'].append(next_space) # update_space_path(next_space)
            print(f"You have moved from {space} to {next_space}!")
            return

    print(f"You navigated through {door} to {new_space}")


#### FUNCTIONs : UPDATE GAME STATE ✅

In [ ]:
# SPACE PATH  -----------------------------------------------------------------------------------------------
# update space path [game room, bedroom 1, bedroom 2...]

def update_space_path(space:str):
    # space is new_space
    while True: # While loop
        if ('door A' in space) and ('door B' not in space) and ('door C' not in space) and ('door D' not in space):
            update_space_path('game room')

        elif ('door A' in space) and ('door B' in space) and ('door C' in space) and ('door D' not in space):
            update_space_path('bedroom 1')
        
        elif ('door A' not in space) and ('door B' in space) and ('door C' not in space) and ('door D' not in space):
            update_space_path('bedroom 2')
        
        elif ('door A' not in space) and ('door B' not in space) and ('door C' in space) and ('door D' in space):
            update_space_path('living room')
        
        elif ('door A' not in space) and ('door B' not in space) and ('door C' not in space) and ('door D' in space):
            update_space_path('outside')
            print("FREEDOM | YOU WIN !")
            print(game_state)
            break
        else:
            print("UNKNOWN SPACE in the system?")
            
        print(f'You are now in {space}')
        update_space_path(space)


# ITEM PATH  -----------------------------------------------------------------------------------------------
# update item path [couch, piano, queen bed...]
def update_item_path(item:str):
    game_state['item path'].append(item)
    return game_state['item path']

# INVENTORY  -----------------------------------------------------------------------------------------------
# update inventory [key door A, key door B, key door C...]
def update_inventory(key:str):
    game_state['inventory'].append(key)
    return game_state['inventory']

# DOOR PATH  -----------------------------------------------------------------------------------------------
# update door path [door A, door B, door C, ...]
def update_door_path(door:str):
    game_state['door path'].append(door)
    return game_state['door path']


    
# UPDATE GAME STATE ----------------------------------------------------------------------------------
def update_game_state(space, item, key, door):
    # update space
    update_space_path(space) 
    # update item
    update_item_path(item) 
    # update inventory with key
    update_inventory(key)
    # update door
    update_door_path(door) 

#### FUNCTION = RESTART() ✅

In [ ]:

# FUNCTION : RESET_GAME --------------------------------------------------------
def reset_game():
    answer = input("Do you want to restart the game? Enter: YES or NO")
    while answer !='YES' and answer!='NO':
        answer = input("To restart Enter only : YES or NO")
    
    if answer.low() == 'yes':
        print("Restarting the game...")
        game_state["space path"].clear()
        game_state["item path"].clear()
        game_state["inventory"].clear()
        game_state["door path"].clear()
    elif answer.low() == 'no':
        print("Continue to play...")
        #player_action('play')
    else:
        print("Value Error: Enter YES or NO")

    return game_state

# RESTART ----------------------------------------------------------------------
def restart():
    reset_game()

#### PLAYER_INPUT = QUIT() ✅

In [ ]:

def quit():
    print("Quitting the game...")
    game_state.clear()
    breakpoint

----
## **GAME PLAY DEMO**

----
## **def MAIN FUNCTION = PLAYER_ACTION(PLAYER_INPUT)**

In [ ]:
def player_action(player_input: str):
     while player_input in actions:
        # QUIT---------------------------------------------------------------------------------------------
        if player_input == "quit":
            print(f"\nPlayer inputs the action: {player_input}")
            print("Quitting the game. Goodbye!")
            break

        # RESTART------------------------------------------------------------------------------------------
        elif player_input == "restart":
            print(f"\nPlayer inputs the action: {player_input}")
            print("Game restarting!")
            # clear the game_state dictionary
            game_state.clear()
            # initialize space="game room"
            update_space_path("game room")
            player_input = 'play'
            player_action(player_input) # Restart automatically

        # PLAY----------------------------------------------------------------------------------------------
        # EXPLORE , EXAMINE, UNLOCK DOOR, NAVIGATE ---------------------------------------------------------                 
        else:
            if game_state.get('game path')==[]: 
                space = 'game room'
            else:
                player_input=input(f"\nEnter an action from: {actions}")
                print(f"Performing current action: {player_input}")
            
            
            # EXPLORE---------------------------------------------------------------------------------------
            if player_input == 'explore':
                trigger_event('space')
                space='game room'
                explore(space)
                
            
            # EXAMINE---------------------------------------------------------------------------------------                          
            elif player_input == 'examine':
                space='game room'
                examine(space)
            
            # UNLOCK DOOR-----------------------------------------------------------------------------------
            elif player_input == 'unlock door':
                print("Doors available to unlock:", doors_in(space))
                door = input("Choose a door to unlock: ")
                inventory = game_state['inventory']
                unlock_door(inventory, space, door)
            # NAVIGATE -------------------------------------------------------------------------------------
            elif player_input == 'navigate':      
                print("Doors available to navigate:", doors_in(space))
                door = input("Select the door to navigate through: ")
                navigate(space, door)
            # LOOP using 'PLAY'  --------------------------------------------------------------------------
            #else:
            #    print("\nInvalid action during play!")
            #    player_input = "play"                        
                        

In [ ]:
# BANNER "WELCOME TO THE ESCAPE ROOM GAME" ---------------------------------------------
print("\n" + "*"*70)
print("*****                WELCOME TO THE ESCAPE ROOM GAME!               *****")
print("*****                      Made by POSH SPICE                       *****")
print("*"*70)

# GAME SCENARIO-------------------------------------------------------------------------
print('''
You wake up on a couch and find yourself in a strange house...
With no windows which you have never been to before...
You don't remember why you are here and what had happened before... 
You feel some unknown danger is approaching...
And you must get out of the house, NOW!
          ''')


# BANNER ----------------------------
game_state['health']=100
print("-" * BANNER_WIDTH)
print(("Welcome to the challengen brave one!"))
print(f"Current Health: {game_state['health']}")
print("-" * BANNER_WIDTH)



# START THE GAME------------------------------------------------------------------------------------------------
print("GAME DEMO!")
print("Starting the game...")
print("-" * BANNER_WIDTH)

# clear game state dict
game_state.clear()
# initialize first space
player_action("play")


---------------
## NOTES
**in navigate, reset countdown and do not reduce HP but stops with take a break at 40min of game play**
